In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
# Función para convertir el ingreso bruto a millones de dólares
def convert_gross_to_millions(gross):
    if gross:
        gross = gross.replace('$', '').replace('M', '').replace(',', '').strip()
        try:
            return float(gross)
        except:
            return None
    return None

# Función principal para realizar el scraping
def scrape_imdb(title_type, user_rating, start_date, end_date):
    base_url = "https://www.imdb.com/search/title/"
    
    # Parámetros de la URL
    params = {
        'title_type': title_type,
        'release_date': f'{start_date},{end_date}',
        'user_rating': f'{user_rating},',
        'countries': 'us',  # Filtra solo por películas de EE. UU.
        'languages': 'en',
        'count': 250  # Máximo de títulos a mostrar por página
    }
    
    # Realiza la solicitud a IMDb
    response = requests.get(base_url, params=params)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Contenedores de películas
    movies_list = soup.find_all('div', class_='lister-item mode-advanced')
    
    # Listas para almacenar los datos
    movie_data = []
    
    # Iterar a través de cada película
    for idx, movie in enumerate(movies_list, start=1):
        # Extraer título
        header = movie.find('h3', class_='lister-item-header')
        title = header.find('a').text
        year = header.find('span', class_='lister-item-year').text
        year = re.sub(r'[^\d]', '', year)  # Extraer solo el año
        
        # Extraer calificación
        rating_tag = movie.find('div', class_='inline-block ratings-imdb-rating')
        rating = float(rating_tag['data-value']) if rating_tag else None
        
        # Extraer duración
        runtime_tag = movie.find('span', class_='runtime')
        runtime = int(runtime_tag.text.replace('min', '').strip()) if runtime_tag else None
        
        # Extraer género
        genre_tag = movie.find('span', class_='genre')
        genre = genre_tag.text.strip() if genre_tag else None
        
        # Extraer descripción
        description_tag = movie.find_all('p', class_='text-muted')
        description = description_tag[1].text.strip() if len(description_tag) > 1 else None
        
        # Extraer director y estrellas
        people = movie.find('p', class_='').text.strip().split('|')
        director = people[0].replace('Director:', '').replace('Directors:', '').strip() if people else None
        stars = people[1].replace('Stars:', '').strip() if len(people) > 1 else None
        
        # Extraer votos
        votes_tag = movie.find('span', {'name': 'nv'})
        votes = int(votes_tag['data-value']) if votes_tag else None
        
        # Extraer ingresos brutos
        gross_tag = movie.find_all('span', {'name': 'nv'})
        gross = convert_gross_to_millions(gross_tag[1]['data-value']) if len(gross_tag) > 1 else None
        
        # Guardar la información en una lista
        movie_data.append({
            'Movie Nr': idx,
            'Title': title,
            'Year': year,
            'Rating': rating,
            'Runtime (min)': runtime,
            'Genre': genre,
            'Description': description,
            'Director': director,
            'Stars': stars,
            'Votes': votes,
            'Gross ($M)': gross
        })
    
    # Convertir los datos a un DataFrame de pandas
    movie_df = pd.DataFrame(movie_data)
    
    return movie_df

# Uso del script con los parámetros solicitados
df = scrape_imdb(title_type='feature', user_rating='7.5', start_date='1990', end_date='1992')

# Mostrar las primeras filas del DataFrame resultante
print(df.head())